In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

os.chdir('/content/gdrive/MyDrive/youtube')

***Номер видео***

In [ ]:
def update_index():
    with open("current_index.txt", "r") as f:
        ind = int(f.read())

    with open("current_index.txt", "w") as f:
        f.write(str(ind + 1))
    return ind
ind = update_index()

***Выбор языка***

In [ ]:
#language = 'en' # en or ru

***Генерируем текст, генерируем ключевые слова и название***

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import random
import ast

with open('openai_api_key.txt', 'r') as file:
    openai.api_key = file.read()

def question(question, model="text-davinci-003"):
    response = openai.Completion.create(
        model=model,
        prompt=f"Human: {question}\nAI:",
        temperature=1,
        max_tokens=2000,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.6,
        stop=[" Human:", " AI:"]
    )
    return response.choices[0].text

def clean_string(s):
    s = s.lstrip()  # Remove leading spaces
    s = s.strip('"').strip("'")  # Remove leading and trailing double quotes
    return s


def generate_texts(sentences=3, filename="content/facts.txt"):
    with open(filename, "r", encoding="utf-8") as f:
        facts = f.read().split("\n")

    selected_facts = random.sample(facts, sentences)
    text = " , , ".join(selected_facts).replace('-', ' ').replace('@', '') # запятую добавил для паузы между фактами, в то время как дефис программа не знает и ломается если его видит, убрал собаку, она все портит и не озвучивается текст

    prompt_description = f"Create an engaging YouTube video description (max 200 characters) for the text '{text}', summarizing content and including relevant keywords."
    text_description = clean_string(question(prompt_description)) #[1:] # убираем пробел из начала текста

    prompt_name = f"Craft a captivating video title (max 80 characters) for: '{text_description}'."
    text_name = clean_string(question(prompt_name))[:100] #[1:][1:-1][:100] # убираем пробел из начала текста # убираем кавычки # укорачиваем название

    prompt_keywords = f"Generate relevant keywords for the video with the description: '{text_description}', separated by commas."
    text_keywords = clean_string(question(prompt_keywords)) #[1:] # убираем пробел из начала текста


    output_folder = f"videos_information/video_{ind}"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Сохранение переменных в файлы
    with open(f"{output_folder}/text.txt", "w", encoding="utf-8") as file:
        file.write(text)

    with open(f"{output_folder}/text_name.txt", "w", encoding="utf-8") as file:
        file.write(text_name)

    with open(f"{output_folder}/text_keywords.txt", "w", encoding="utf-8") as file:
        file.write(text_keywords)

    with open(f"{output_folder}/text_description.txt", "w", encoding="utf-8") as file:
        file.write(text_description)

    return text, text_name, text_keywords, text_description, selected_facts

text, text_name, text_keywords, text_description, selected_facts = generate_texts()

In [ ]:
#print(text)
#print(text_name)
#print(text_keywords)
#print(text_description)

***Создаем папку для нашего видео***

In [ ]:
#video_folder = f"videos_information/video_{ind}"
#os.makedirs(video_folder, exist_ok=True)  # creates directory if it does not exist

***Озвучиваем текст***

In [ ]:
os.chdir('/content/gdrive/MyDrive/youtube')

In [ ]:
!pip install g2p_en nemo_text_processing soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 119.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 9

In [ ]:
#os.chdir('/content/gdrive/MyDrive/youtube')
#!ls

In [ ]:
import ipywidgets as widgets
import IPython
import json
from IPython.core.display import display
import soundfile as sf


global speaker, talking_speed, text, button_clear, output_clear, button, output


def synth_audio():
    global tts
    %cd /content
    try:
        tts
    except NameError:
        print()
        print("Starting initial download, this can take a minute and will only be required once ...")
        !git clone https://github.com/dunky11/deep-ponies-frontend
        %cd /content/deep-ponies-frontend
        !pip install -r requirements.txt
        !pip install -U --no-cache-dir gdown --pre
        from api import download_dependencies, DeepPoniesTTS
        download_dependencies()
        tts = DeepPoniesTTS()
    %cd /content/gdrive/MyDrive/youtube

    print()
    print("Converting audio ...")
    audio = tts.synthesize(text, speaker, 1 / talking_speed)
    IPython.display.clear_output()
    print()
    display(IPython.display.Audio(audio, rate=22050))

    # Сохранение аудиофайла в папке video_folder
    os.chdir('/content/gdrive/MyDrive/youtube')
    output_file = os.path.join(video_folder, "speech.wav")
    sf.write(output_file, audio, 22050, subtype='PCM_16')
    return output_file


#speaker = 'Nameless Hero' #@param ['Adachi Tohru', 'Apple Bloom', 'Applejack', 'Barack Obama', 'Bart Simpson', 'Billie Eilish', 'Celestia', 'Chie Satonaka', 'Cozy Glow', 'Demoman', 'Discord', 'Donald Trump', 'Engineer', 'Fluttershy', 'Franklin', 'GLaDOS', 'Granny Smith', 'Heavy', 'Homer Simpson', 'Joe Biden', 'Joe Rogan', 'Kanji Tatsumi', 'Kanye West', 'Kim Kardashian', 'Kratos', 'Luna', 'Medic', 'Michael', 'Nameless Hero', 'Nanako Dojima', 'Naoto Shirogane', 'Pinkie Pie', 'Rainbow Dash', 'Rarity', 'Rise Kujikawa', 'Ryotaro Dojima', 'Scootaloo', 'Scout', 'Sniper', 'Soldier', 'Spike', 'SpongeBob', 'Spy', 'Starlight', 'Sunset Shimmer', 'Sweetie Belle', 'Teddie', 'Trevor', 'Trixie', 'Twilight Sparkle', 'Yosuke Hanamura', 'Yukiko Amagi']
#talking_speed = 1.00 #@param {type:"slider", min: 0.1, max:2.0, step:0.05}

#destination_voice, audio = synth_audio()

***Генерируем картинки***

In [ ]:
!pip install Pillow
!wget https://github.com/liberationfonts/liberation-fonts/files/7042832/LiberationSans-Regular.ttf

--2023-07-03 11:19:32--  https://github.com/liberationfonts/liberation-fonts/files/7042832/LiberationSans-Regular.ttf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-03 11:19:32 ERROR 404: Not Found.



In [ ]:
#images_folder = os.path.join(video_folder, "images")
#if not os.path.exists(images_folder):
#    os.makedirs(images_folder)

In [ ]:
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import base64
import re
import random
import numpy as np


#with open('pixabey_api_key.txt', 'r') as file:
#    API_KEY = file.read()
#fixed_width = 900
#fixed_height = 630

#api_lim = 0


def is_grey_scale(img_path):
    """Возвращает True, если изображение монохромное."""
    img = Image.open(BytesIO(requests.get(img_path).content)).convert('RGB')
    img.thumbnail((10, 10))  # Уменьшаем размер изображения для ускорения проверки
    arr = np.array(img)
    if np.array_equal(arr[..., 0], arr[..., 1]) and np.array_equal(arr[..., 0], arr[..., 2]):
        return True
    else:
        return False

def get_image_url(search_query):
    global api_lim, fixed_width, fixed_height, API_KEY
    api_lim += 1
    url = f"https://pixabay.com/api/?key={API_KEY}&q={search_query}&image_type=photo&orientation=horizontal&min_width={fixed_width}&max_width={fixed_width}&safesearch=true"
    if api_lim == 99:
        api_lim = 0
        time.sleep(60)
    response = requests.get(url)
    data = response.json()
    if 'totalHits' in data and data['totalHits'] > 0:
        for hit in random.sample(data['hits'][:5], len(data['hits'][:5])): #for hit in data['hits']:
            img_url = hit['webformatURL']
            if not is_grey_scale(img_url):
                return img_url
    return None


def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return f'data:image/jpeg;base64,{img_str}'

def resize_image(image, target_width):
    width, height = image.size
    aspect_ratio = float(height) / float(width)
    new_height = int(target_width * aspect_ratio)
    resized_image = image.resize((target_width, new_height), Image.ANTIALIAS)
    return resized_image

def save_image(word, image, target_width=480):
    resized_image = resize_image(image, target_width)

    if resized_image.mode == 'RGBA':
        resized_image = resized_image.convert('RGB')

    img_path = os.path.join(images_folder, f"{word}.jpg")
    resized_image.save(img_path)
    return img_path

def remove_punctuation(text):
    result = re.sub(r'[^\w\s]', '', text)
    return result

def generate_images():
    words = text.split()
    words = [word for word in words if word != ',']

    print(words)
    destination_images = []
    words_for_img = 2

    for i, word in enumerate(words):
        prev_word, next_word = '', ''
        if i - 1 >= 0:
            prev_word = words[i - 1]
        image_url = get_image_url(remove_punctuation(prev_word + ' ' + word))
        if image_url:
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            img_path = save_image(word, img)
            #print(f"Изображение для поискового запроса '{word}' сохранено в: {img_path}")
        else:
            img_path = 'NO_PIC'
            print(f"Картинка с текстом '{word}' сохранена в: {img_path}")

        destination_images.append(img_path)

    if destination_images[0] == 'NO_PIC':
        i = 1
        while destination_images[i] == 'NO_PIC':
            i += 1
        destination_images[0] = destination_images[i]

    for i, dest in enumerate(destination_images):
        if destination_images[i] == 'NO_PIC':
            destination_images[i] = destination_images[i - 1]

    return destination_images

#text = 'The word “white” comes from the Indo European root “kweit” meaning “to shine.” , , Iceland does not have a railway system. , , Garlic is known to attract leeches. , , The logo for the Monster energy drink cans is not an M, as most believe, but are claw marks from a monster ripping through the can. In 2014 a YouTube video gained an international audience when they claimed that each segment of the claw mark looks like the Hebrew symbol for the number six, making the logo into 666, the number of the Beast.'
#with open('pixabey_api_key.txt', 'r') as file:
#    API_KEY = file.read()#fixed_width = 900
#fixed_height = 630

#api_lim = 0
#destination_images = generate_images()

***Находим/генерируем музыку***

In [ ]:
def get_random_track(folder):
    tracks = [f for f in os.listdir(folder) if f.endswith('.mp3')]
    return os.path.join(folder, random.choice(tracks))

def generate_music():
    random_track = get_random_track('music')
    print(f'Playing random track: {random_track}')

    track_name = os.path.basename(random_track)
    destination_track = os.path.join(video_folder, track_name)
    shutil.copy2(random_track, destination_track)

    return destination_track

#destination_track = generate_music()

***Находим/генерируем фон***

In [ ]:
def generate_background():
    background_folder = "backgrounds" # no_sound_backgrounds

    background_files = [f for f in os.listdir(background_folder) if f.endswith(('.mp4', '.avi', '.mkv', '.mov', 'MOV'))]

    background = os.path.join(background_folder, random.choice(background_files))

    print(f"Выбранное видео: {background}")

    # Save the selected video in the same location
    background_name = os.path.basename(background)
    destination_background = os.path.join(video_folder, background_name)
    shutil.copy(background, destination_background)

    return destination_background

#destination_background = generate_background()

***Обьединяем это все + Субтитры***

In [ ]:
!pip install srt
!pip install pyphen
!sudo pacman -S ffmpeg
#!pip uninstall whisper
!pip install git+https://github.com/openai/whisper.git

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=48cc1dd9eb6f6402296d1ef8328b61efc3db7ca197a601d59c4063d95fea93f7
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.5 MB/s eta 0:00:00
sudo: pacman: command not found
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pc6hi092
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pc6hi092
  Resolved https://github.com/openai/whisper.git to commit f572f2161ba831bae131364c3bffdead7af6d210
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-

In [ ]:
import os
import datetime
import srt
import whisper
from moviepy.editor import *
import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects
from PIL import Image
import io
import numpy as np
import pyphen

def count_syllables(word):
    dic = pyphen.Pyphen(lang='en')
    syllables = dic.inserted(word)
    syllable_count = syllables.count('-') + 1

    return syllable_count

def transcribe_audio_to_srt(audio_path):
    model = whisper.load_model("base.en")
    result = model.transcribe(audio_path)

    subtitles = []
    subtitle_index = 1
    start_time = 0

    for segment in result["segments"]:
        start = start_time + segment["start"]
        end = start_time + segment["end"]
        duration = end - start

        words = segment["text"].split()
        total_word_duration = sum(count_syllables(word) for word in words)
        word_durations = [duration * count_syllables(word) / total_word_duration for word in words]

        current_time = start

        for i, word in enumerate(words):
            word_start = current_time
            word_end = current_time + word_durations[i]

            subtitle = srt.Subtitle(index=subtitle_index, start=datetime.timedelta(seconds=word_start), end=datetime.timedelta(seconds=word_end), content=word)
            subtitles.append(subtitle)
            subtitle_index += 1

            current_time = word_end

        start_time += segment["end"]

    max_gap = 1
    reduced_gap = 0.0

    for i in range(1, len(subtitles)):
        gap = subtitles[i].start - subtitles[i - 1].end
        if gap.total_seconds() > max_gap:
            subtitles[i].start -= datetime.timedelta(seconds=gap.total_seconds() - reduced_gap)
            subtitles[i].end -= datetime.timedelta(seconds=gap.total_seconds() - reduced_gap)

    return srt.compose(subtitles)


def create_word_image(text, fontsize=100, color='white', bgcolor='black', font="sans-serif"): # подгони fontsize под текущий размер видео попробуй 100 и 50
    fig, ax = plt.subplots()
    text_artist = ax.text(0.5, 0.5, text, fontsize=fontsize, color=color, fontweight='bold', fontname=font, ha='center', va='center')
    text_artist.set_path_effects([patheffects.Stroke(linewidth=20, foreground='black'), patheffects.Normal()])  # Увеличиваем значение linewidth для толще рамки
    ax.set_facecolor(bgcolor)
    plt.axis('off')
    fig.tight_layout(pad=0)

    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', transparent=True, dpi=100, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    buffer.seek(0)
    return buffer


def create_subtitle_clips(subtitle_file, video_clip, fontsize=100, color='white', stroke_color='black', stroke_width=1, max_words=5, max_width_ratio=0.8):
    subtitle_clips = []

    with open(subtitle_file, "r") as f:
        srt_content = f.read()

    for subtitle in srt.parse(srt_content):
        group_start_time = subtitle.start.total_seconds()
        group_end_time = subtitle.end.total_seconds()
        group_sentence = subtitle.content

        max_width = int(video_clip.size[0] * max_width_ratio)
        buffer = create_word_image(group_sentence, fontsize=fontsize, color=color, bgcolor='black', font="sans-serif")
        img = Image.open(buffer)
        img.thumbnail((max_width, video_clip.size[1]), Image.ANTIALIAS)
        np_img = np.array(img)
        txt = ImageClip(np_img).set_start(group_start_time).set_end(group_end_time).set_position(('center', 'center'))
        subtitle_clips.append(txt)

    return subtitle_clips

def match_list_length(longer_list, shorter_list):
    difference = len(longer_list) - len(shorter_list)

    if difference > 0:  # Если короче, дополните список значениями последнего элемента
        last_element = shorter_list[-1]
        shorter_list.extend([last_element] * difference)
    elif difference < 0:  # Если длиннее, обрежьте список
        shorter_list = shorter_list[:len(longer_list)]

    return shorter_list

def union_all(destination_voice, destination_background, destination_track, img_destinations):
    video_clip = VideoFileClip(destination_background)
    audio_clip = AudioFileClip(destination_track)
    voice_clip = AudioFileClip(destination_voice)

    total_duration = voice_clip.duration + 1

    video_clip = video_clip.set_duration(total_duration)
    audio_clip = audio_clip.volumex(0.1).set_duration(total_duration)

    mixed_audio = CompositeAudioClip([audio_clip, voice_clip])

    srt_content = transcribe_audio_to_srt(destination_voice)

    with open("subtitles.srt", "w") as f:
        f.write(srt_content)

    subtitle_clips = create_subtitle_clips("subtitles.srt", video_clip)

    subtitles = list(srt.parse(srt_content))

    image_clips = []

    for i, img_destination in enumerate(match_list_length(subtitles, img_destinations)):
        start_time = subtitles[i].start.total_seconds()
        end_time = subtitles[i].end.total_seconds()

        image_clip = ImageClip(img_destination).set_position(('center', 100))
        image_clip = image_clip.set_start(start_time).set_end(end_time)
        image_clips.append(image_clip)

    final_clip = CompositeVideoClip([video_clip] + image_clips + subtitle_clips).set_audio(mixed_audio)

    output_folder = "videos"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    final_clip.write_videofile(
        f"{output_folder}/output_video_{ind}.mp4",
        codec="libx264",
        audio_codec="aac",
        threads=4,
        logger="bar"
    )

#union_all(destination_voice, destination_background, destination_track, destination_images)


































***Выкладываем на ютуб***

In [ ]:
#!pip install pytz

In [ ]:
import google.oauth2.credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
import json
import pytz

def get_publish_time():
    file_path = "publish_time.txt"
    publish_hours = [19, 23, 5]  # Define the times of the day to publish
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            publish_time = datetime.datetime.strptime(file.read(), "%Y-%m-%dT%H:%M:%SZ")
    else:
        # set publish_time to midnight of the next day
        now = datetime.datetime.now(pytz.timezone('UTC'))
        publish_time = now.replace(hour=0, minute=0, second=0, microsecond=0) + datetime.timedelta(days=1)

    # update publish_time for the next video
    if publish_time.hour == 19:
        next_publish_time = publish_time + datetime.timedelta(hours=4)
    elif publish_time.hour == 23:
        next_publish_time = publish_time + datetime.timedelta(hours=6)
    elif publish_time.hour == 5:
        next_publish_time = publish_time + datetime.timedelta(hours=14)
    with open(file_path, 'w') as file:
        file.write(next_publish_time.strftime("%Y-%m-%dT%H:%M:%SZ"))

    return publish_time.strftime("%Y-%m-%dT%H:%M:%SZ")

def get_authenticated_service():
    credentials_file = "credentials_facts.json"
    client_secrets_file = "client_secret_facts.json"
    scopes = ["https://www.googleapis.com/auth/youtube.upload"]

    if os.path.exists(credentials_file):
        with open(credentials_file, 'r') as f:
            creds_data = json.load(f)
        credentials = google.oauth2.credentials.Credentials.from_authorized_user_info(info=creds_data, scopes=scopes)
    else:
        flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes=scopes)
        credentials = flow.run_local_server(port=8081)
        with open(credentials_file, 'w') as f:
            json.dump(json.loads(credentials.to_json()), f)

    return build("youtube", "v3", credentials=credentials)

def upload_video(youtube, video_file, title, description, tags):
    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags,
            "categoryId": "24"
        },
        "status": {
            "privacyStatus": "private",
            "madeForKids": False,
            "publishAt": get_publish_time()
        }
    }

    media = MediaFileUpload(video_file, chunksize=-1, resumable=True, mimetype="video/*")
    request = youtube.videos().insert(
        part=",".join(body.keys()),
        body=body,
        media_body=media
    )

    response = None
    while response is None:
        status, response = request.next_chunk()
        if status:
            print(f"Uploaded {int(status.progress() * 100)}.")
    print(f"Upload Complete!")

'''
if __name__ == "__main__":
    try:
        youtube = get_authenticated_service()
        upload_video(youtube, f"videos/output_video_{ind}.mp4", text_name, text_description, text_keywords)
    except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred: {e.content}")
'''

'\nif __name__ == "__main__":\n    try:\n        youtube = get_authenticated_service()\n        upload_video(youtube, f"videos/output_video_{ind}.mp4", text_name, text_description, text_keywords)\n    except HttpError as e:\n        print(f"An HTTP error {e.resp.status} occurred: {e.content}")\n'

***Бесконечный запуск***

In [ ]:
import time
import shutil

for i in range(1):
    ind = update_index()
    language = 'en'
    with open('openai_api_key.txt', 'r') as file:
        openai.api_key = file.read()
    video_folder = f"videos_information/video_{ind}"

    text, text_name, text_keywords, text_description, selected_facts = generate_texts(sentences=3, filename="content/facts.txt")

    speaker = 'Starlight'
    talking_speed = 1.00
    destination_voice = synth_audio()

    images_folder = os.path.join(video_folder, "images")
    print(video_folder, images_folder)
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    with open('pixabey_api_key.txt', 'r') as file:
        API_KEY = file.read()
    fixed_width = 900 #480
    fixed_height = 630 #335
    api_lim = 0
    destination_images = generate_images()

    destination_track = generate_music()
    destination_background = generate_background()
    union_all(destination_voice, destination_background, destination_track, destination_images)
    try:
        youtube = get_authenticated_service()
        upload_video(youtube, f"videos/output_video_{ind}.mp4", text_name, text_description, text_keywords)
    except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred: {e.content}")

    shutil.rmtree(video_folder)

videos_information/video_667 videos_information/video_667/images
['Sprite', 'was', 'manufactured', 'as', 'Coca', 'Cola’s', 'response', 'to', 'the', 'popularity', 'of', '7', 'Up.', '3', 'Million', 'Americans', 'still', 'subscribed', 'to', 'AOL', 'Dial', 'Up', 'Internet.', 'The', 'American', 'one', 'has', '2', 'green', 'spaces', 'while', 'the', 'European', 'one', 'only', 'has', '1.']
Картинка с текстом 'was' сохранена в: NO_PIC
Картинка с текстом 'manufactured' сохранена в: NO_PIC
Картинка с текстом 'the' сохранена в: NO_PIC
Картинка с текстом 'Americans' сохранена в: NO_PIC
Картинка с текстом 'subscribed' сохранена в: NO_PIC
Картинка с текстом 'AOL' сохранена в: NO_PIC
Картинка с текстом 'Dial' сохранена в: NO_PIC
Картинка с текстом '2' сохранена в: NO_PIC
Картинка с текстом 'while' сохранена в: NO_PIC
Картинка с текстом 'has' сохранена в: NO_PIC
Картинка с текстом '1.' сохранена в: NO_PIC
Playing random track: music/Volfgang-Amadey-Mocart_-_Malenkaya-nochnaya-serenada-1-ch_(muziqa.ru).

100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 10.7MiB/s]


Moviepy - Building video videos/output_video_667.mp4.
MoviePy - Writing audio in output_video_667TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/output_video_667.mp4



Moviepy - Done !
Moviepy - video ready videos/output_video_667.mp4
Upload Complete!
